In [27]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

val_image_path = './training_all'
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = ('soccer/notebooks/'+val_image_path + '/' + root.find('filename').text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text),
                    #  int(root.find('size')[0].text),
                    #  int(root.find('size')[1].text),
                     member[0].text
                     )
            xml_list.append(value)
    column_name = ['path_to_image', 'xmin', 'ymin', 'xmax', 'ymax', 'class']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), 'Annotations')
    xml_df = xml_to_csv(val_image_path)
    xml_df.to_csv('annotations.csv', index=None)
    print('Successfully converted xml to csv.')


main()

Successfully converted xml to csv.


In [7]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import csv

def generate_class_mapping():
    class_map_list = []
    fieldnames =  ['path_to_image','xmin','ymin','xmax','ymax','class']
    i = 0
    with open('./annotations.csv', 'r') as csv_file:
        reader = csv.DictReader(csv_file, fieldnames=fieldnames)
        next(reader)
        for line, row in enumerate(reader):
            #print(row['class'])
            value = (row['class'], i)
            i += 1
            class_map_list.append(value)
    column_name = ['class', 'id']
    df = pd.DataFrame(class_map_list, columns=column_name)
    return df


def main():
    df = generate_class_mapping()
    df.to_csv('class_mappings.csv', index=None)
    # image_path = os.path.join(os.getcwd(), 'Annotations')
    # xml_df = xml_to_csv(image_path)
    # xml_df.to_csv('annotations.csv', index=None)
    # print('Successfully converted xml to csv.')


main()

In [25]:
import numpy as np
df = generate_class_mapping()
df = pd.unique(df['class'])
df = pd.DataFrame({'class':df,'id':np.arange(len(df))})
df.head()

,class,id
0,player,0
1,ball,1
2,referee,2
3,goalie,3
4,manager,4


In [26]:
df.to_csv('class_mappings.csv', index=None)

In [28]:
from tqdm import tqdm


In [29]:
import fastprogress

In [30]:
from fastprogress import master_bar, progress_bar
from time import sleep
mb = master_bar(range(10))

for i in range(0,100):
    for j in range(0,50):
        sleep(0.001)
    mb.update(i)
        

In [32]:
from fastprogress import master_bar, progress_bar
from time import sleep
mb = master_bar(range(10))
for i in mb:
    for j in progress_bar(range(100), parent=mb):
        sleep(0.01)
        mb.child.comment = f'second bar stat'
    mb.first_bar.comment = f'first bar stat'
    mb.write(f'Finished loop {i}.')

TypeError: __init__() got an unexpected keyword argument 'display'

In [1]:
!pwd

/home/ewaymao/retinanet-pytorch-1.0/soccer/notebooks


In [2]:
%cd ../..

/home/ewaymao/retinanet-pytorch-1.0


In [3]:
import numpy as np
%matplotlib inline
import torchvision
import time
import os
import copy
import pdb
import time
import argparse

import sys
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer


#assert torch.__version__.split('.')[1] == '4'

print('CUDA available: {}'.format(torch.cuda.is_available()))


def main(args=None):
    parser = argparse.ArgumentParser(description='Simple training script for training a RetinaNet network.')

    #parser.add_argument('--dataset', help='Dataset type, must be one of csv or coco.')
    #parser.add_argument('--coco_path', help='Path to COCO directory')
    #parser.add_argument('--csv_classes', help='Path to file containing class list (see readme)')
    #parser.add_argument('--csv_val', help='Path to file containing validation annotations (optional, see readme)')

    #parser.add_argument('--model', help='Path to model (.pt) file.')
    
    #parser = parser.parse_args(args)
    parser.dataset = 'csv'
    parser.csv_classes = 'soccer/notebooks/class_mappings.csv'
    parser.csv_val = 'soccer/notebooks/annotations.csv'
    parser.model = 'model_final.pt'
    
    if parser.dataset == 'coco':
        dataset_val = CocoDataset(parser.coco_path, set_name='val2017', transform=transforms.Compose([Normalizer(), Resizer()]))
    elif parser.dataset == 'csv':
        dataset_val = CSVDataset(train_file=parser.csv_val, class_list=parser.csv_classes, transform=transforms.Compose([Normalizer(), Resizer()]))
    else:
        raise ValueError('Dataset type not understood (must be csv or coco), exiting.')

    sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
    dataloader_val = DataLoader(dataset_val, num_workers=1, collate_fn=collater, batch_sampler=sampler_val)

    retinanet = torch.load(parser.model)
    #retinanet = model.resnet50(num_classes=80,)
    #retinanet.load_state_dict(torch.load('coco_resnet50_statedict.pt'))

    retinanet.eval()

    unnormalize = UnNormalizer()

    def draw_caption(image, box, caption):

        b = np.array(box).astype(int)
        cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
        cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)


    

    for idx, data in enumerate(dataloader_val):

        with torch.no_grad():
            st = time.time()
            scores, classification, transformed_anchors = retinanet(data['img'].float())
            print('Elapsed time: {}'.format(time.time()-st))

            #breakpoint()
            if idx >= 1: break
            
            idxs = np.where(scores>0.1)                                                    
            img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()

            img[img<0] = 0
            img[img>255] = 255

            img = np.transpose(img, (1, 2, 0))

            img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)

            for j in range(idxs[0].shape[0]):
                bbox = transformed_anchors[idxs[0][j], :]
                x1 = int(bbox[0])
                y1 = int(bbox[1])
                x2 = int(bbox[2])
                y2 = int(bbox[3])
                label_name = dataset_val.labels[int(classification[idxs[0][j]])]
                draw_caption(img, (x1, y1, x2, y2), label_name)

                cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)
                print(label_name)

            cv2.imshow('img', img)
            cv2.waitKey(0)



if __name__ == '__main__':
    main()

CUDA available: False
Elapsed time: 2.373469591140747
Elapsed time: 2.402557611465454


In [1]:
!pwd

/home/ewaymao/retinanet-pytorch-1.0/soccer/notebooks


In [3]:
!ls training_all/

10801.jpg  16531.xml  22621.jpg  3501.xml  4251.jpg  5801.xml  8701.jpg
10801.xml  17401.jpg  22621.xml  3601.jpg  4251.xml  6091.jpg  8701.xml
14211.jpg  17401.xml  241.jpg	 3601.xml  4641.jpg  6091.xml  8881.jpg
14211.xml  17691.jpg  241.xml	 3751.jpg  4641.xml  6381.jpg  8881.xml
14501.jpg  17691.xml  2611.jpg	 3751.xml  4931.jpg  6381.xml  8991.jpg
14501.xml  17981.jpg  2611.xml	 3841.jpg  4931.xml  6961.jpg  8991.xml
14791.jpg  17981.xml  2751.jpg	 3841.xml  5281.jpg  6961.xml  9281.jpg
14791.xml  18271.jpg  2751.xml	 4001.jpg  5281.xml  7831.jpg  9281.xml
15081.jpg  18271.xml  3001.jpg	 4001.xml  5511.jpg  7831.xml  9571.jpg
15081.xml  2001.jpg   3001.xml	 4061.jpg  5511.xml  8411.jpg  9571.xml
15371.jpg  2001.xml   3251.jpg	 4061.xml  5751.jpg  8411.xml
15371.xml  20881.jpg  3251.xml	 4081.jpg  5751.xml  8641.jpg
16531.jpg  20881.xml  3501.jpg	 4081.xml  5801.jpg  8641.xml


In [5]:
!ls training_all/ | grep xml > xml_list

In [6]:
!cat xml_list

10801.xml
14211.xml
14501.xml
14791.xml
15081.xml
15371.xml
16531.xml
17401.xml
17691.xml
17981.xml
18271.xml
2001.xml
20881.xml
22621.xml
241.xml
2611.xml
2751.xml
3001.xml
3251.xml
3501.xml
3601.xml
3751.xml
3841.xml
4001.xml
4061.xml
4081.xml
4251.xml
4641.xml
4931.xml
5281.xml
5511.xml
5751.xml
5801.xml
6091.xml
6381.xml
6961.xml
7831.xml
8411.xml
8641.xml
8701.xml
8881.xml
8991.xml
9281.xml
9571.xml


In [1]:
import xml

In [3]:
import os
import xml.etree.ElementTree as ET
import xmltodict
import json
from xml.dom import minidom
from collections import OrderedDict


#attrDict = {"images":[{"file_name":[],"height":[], "width":[],"id":[]}], "type":"instances", "annotations":[], "categories":[]}

#xmlfile = "000023.xml"


def generateVOC2Json(rootDir,xmlFiles):
    #breakpoint()
    attrDict = dict()
    #images = dict()
    #images1 = list()
    attrDict["categories"]=[{"supercategory":"none","id":0,"name":"player"},
                    {"supercategory":"none","id":1,"name":"ball"},
                    {"supercategory":"none","id":2,"name":"referee"},
                    {"supercategory":"none","id":3,"name":"goalie"},
                {"supercategory":"none","id":4,"name":"manager"},
                {"supercategory":"none","id":5,"name":"cameraman"}]
    images = list()
    annotations = list()
    for root, dirs, files in os.walk(rootDir):
        image_id = 0
        for file in xmlFiles:
            image_id = image_id + 1
            
            #breakpoint()
            if file in files:
                
                #image_id = image_id + 1
                annotation_path = os.path.abspath(os.path.join(root, file))
                
                #tree = ET.parse(annotation_path)#.getroot()
                image = dict()
                #keyList = list()
                doc = xmltodict.parse(open(annotation_path).read())
                #print doc['annotation']['filename']
                image['file_name'] = str(doc['annotation']['filename'])
                #keyList.append("file_name")
                image['height'] = int(doc['annotation']['size']['height'])
                #keyList.append("height")
                image['width'] = int(doc['annotation']['size']['width'])
                #keyList.append("width")

                #image['id'] = str(doc['annotation']['filename']).split('.jpg')[0]
                image['id'] = image_id
                print("File Name: {} and image_id {}".format(file, image_id))
                images.append(image)
                # keyList.append("id")
                # for k in keyList:
                #     images1.append(images[k])
                # images2 = dict(zip(keyList, images1))
                # print images2
                #print images

                #attrDict["images"] = images

                #print attrDict
                #annotation = dict()
                id1 = 1
                if 'object' in doc['annotation']:
                    for obj in doc['annotation']['object']:
                        for value in attrDict["categories"]:
                            annotation = dict()
                            #if str(obj['name']) in value["name"]:
                            if str(obj['name']) == value["name"]:
                                #print str(obj['name'])
                                #annotation["segmentation"] = []
                                annotation["iscrowd"] = 0
                                #annotation["image_id"] = str(doc['annotation']['filename']).split('.jpg')[0] #attrDict["images"]["id"]
                                annotation["image_id"] = image_id
                                x1 = int(obj["bndbox"]["xmin"])  - 1
                                y1 = int(obj["bndbox"]["ymin"]) - 1
                                x2 = int(obj["bndbox"]["xmax"]) - x1
                                y2 = int(obj["bndbox"]["ymax"]) - y1
                                annotation["bbox"] = [x1, y1, x2, y2]
                                annotation["area"] = float(x2 * y2)
                                annotation["category_id"] = value["id"]
                                annotation["ignore"] = 0
                                annotation["id"] = id1
                                annotation["segmentation"] = [[x1,y1,x1,(y1 + y2), (x1 + x2), (y1 + y2), (x1 + x2), y1]]
                                id1 +=1

                                annotations.append(annotation)
                
                else:
                    print("File: {} doesn't have any object".format(file))
                #image_id = image_id + 1
                
            else:
                print("File: {} not found".format(file))
            

    attrDict["images"] = images    
    attrDict["annotations"] = annotations
    attrDict["type"] = "instances"

    #print attrDict
    jsonString = json.dumps(attrDict)
    with open("soccer_annotations.json", "w") as f:
        f.write(jsonString)

# rootDir = "/netscratch/pramanik/OBJECT_DETECTION/detectron/lib/datasets/data/Receipts/Annotations"
# for root, dirs, files in os.walk(rootDir):
#     for file in files:
#         if file.endswith(".xml"):
#             annotation_path = str(os.path.abspath(os.path.join(root,file)))
#             #print(annotation_path)
#             generateVOC2Json(annotation_path)
trainFile = "xml_list"
trainXMLFiles = list()
with open(trainFile, "rb") as f:
    #breakpoint()
    for line in f:
        fileName = line.strip().decode('ascii')
        print(fileName)
        trainXMLFiles.append(fileName)


rootDir = "./training_all/"
generateVOC2Json(rootDir, trainXMLFiles)

> <ipython-input-3-b13a5db17d03>(119)<module>()
-> for line in f:
(Pdb) n
> <ipython-input-3-b13a5db17d03>(120)<module>()
-> fileName = line.strip().decode('ascii')
(Pdb) c
10801.xml
14211.xml
14501.xml
14791.xml
15081.xml
15371.xml
16531.xml
17401.xml
17691.xml
17981.xml
18271.xml
2001.xml
20881.xml
22621.xml
241.xml
2611.xml
2751.xml
3001.xml
3251.xml
3501.xml
3601.xml
3751.xml
3841.xml
4001.xml
4061.xml
4081.xml
4251.xml
4641.xml
4931.xml
5281.xml
5511.xml
5751.xml
5801.xml
6091.xml
6381.xml
6961.xml
7831.xml
8411.xml
8641.xml
8701.xml
8881.xml
8991.xml
9281.xml
9571.xml
File Name: 10801.xml and image_id 1
File Name: 14211.xml and image_id 2
File Name: 14501.xml and image_id 3
File Name: 14791.xml and image_id 4
File Name: 15081.xml and image_id 5
File Name: 15371.xml and image_id 6
File Name: 16531.xml and image_id 7
File Name: 17401.xml and image_id 8
File Name: 17691.xml and image_id 9
File Name: 17981.xml and image_id 10
File Name: 18271.xml and image_id 11
File Name: 2001.xml a